#Import Modules

In [ ]:
#0 code flow

#A historical data collection
#1 check if sql directory and store exists
#2 if 1 is true, check if any part of date range of required data exists
#3 if 2 is true, extract data and extrapolate missing data
#4 if 2 not true, get whole historical data from internet
#5 perform operations on df from internet
#6 add new info to sql store

#B websocket live data collection
#1 get missing latest data date range
#2 extract data live  from websocket
#3 perform operations on df from internet
#4 add new info to sql store


In [ ]:
%pip install python-binance
%pip install mplfinance
%pip install wget
%pip install python-decouple


In [ ]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline 
 
import sys
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from importlib import reload

import mplfinance as mpf
import matplotlib.pyplot as plt

sys.path.insert(1, './logic')  
import helperfunctions as hlpf
import gethistoricaldf as ghdf
reload(hlpf)
reload(ghdf)

In [ ]:
#get all list containing required data for use
all_lst = ghdf.getdf()
symbols_lst, intrvl_lst, df_lst, df_ochlv_lst = zip(*all_lst)


In [ ]:
# quick check of data unpacked
print(f'nos of symbolpairs: {len(symbols_lst)}')
print(f'nos of intervals: {len(intrvl_lst)}')
print(f"nos of full df's: {len(df_lst)}")
print(f"nos of ochlv df's: {len(df_ochlv_lst)}")

print(f'list of symbolpairs: {symbols_lst}')
print(f'list of intervals: {intrvl_lst}')

for cnt, (symbol, intrvl, df_full, df_ochlv) in enumerate(all_lst):
    print(f'count: {cnt}|symbol: {symbol}|interval:{intrvl}|rows orig df:{len(df_full.index)}|rows ochlv df:{len(df_ochlv.index)}')


#add indicators and signals to df

In [ ]:
# df analysis and new column additions for the ochlv df

#output matplotlib chart

In [ ]:
#i need to dynamically access this dates data to transform the plot!!!!!
#this needs to be done differently for every type of interval
datapts_btw_start_end = 200
datapts_shift = 0 #this usually same as datapts_btw_start_end
datapts_shift_factor = 1 #can be a 1,2,3,4 etc

date_from_lst, date_to_lst = hlpf.getsliced_date_lsts(intrvl_lst, datapts_btw_start_end, datapts_shift, datapts_shift_factor)


In [ ]:
slicedate_lst = hlpf.get_sliced_df(symbols_lst, intrvl_lst, date_from_lst, date_to_lst, df_lst, df_ochlv_lst)
symbols_lst, intrvl_lst, df_full_slicedate_lst, df_ochlv_slicedate_lst, df_lst, df_ochlv_lst = zip(*slicedate_lst)

In [ ]:
#settings for the matplotlib  plot
show_nontrading_days = True
chart_styles = [
                'default', 'binance', 'blueskies', 'brasil', 
                'charles', 'checkers', 'classic', 'yahoo',
                'mike', 'nightclouds', 'sas', 'starsandstripes'
            ]
chart_style = chart_styles[1] #let this be selected from chart_styles instead

chart_types = ['candle', 'ohlc', 'line', 'renko', 'pnf']
chart_type = chart_types[0]


#indicator settings
#need to correct this properly
mav1 = 3 #st.number_input('Mav 1', min_value=3, max_value=30, value=3, step=1)
mav2 = 6 #st.number_input('Mav 2', min_value=3, max_value=30, value=6, step=1)
mav3 = 9 #st.number_input('Mav 3', min_value=3, max_value=30, value=9, step=1)
mavset=(int(mav1),int(mav2),int(mav3))


In [ ]:
#list all data going into plot
for cnt, (symbol, intrvl, df_full_slicedate, df_ochlv_slicedate, df_full, df_ochlv) in enumerate(slicedate_lst): 
    print("=" * 50)
    print(f'count: {cnt}')
    print(f'symbol: {symbol}|date_from: {date_from}|date_to: {date_to}|intrvl: {intrvl}')
    print(f'chart_type: {chart_type}|show_nontrading_days: {show_nontrading_days}|mavset: {mavset}|chart_style: {chart_style}')
   
    hlpf.plotchart(df_ochlv_slicedate, symbol, date_from, date_to, intrvl, chart_type, show_nontrading_days, mavset, chart_style)